In [56]:
import tensorflow as tf 

In [102]:
import ampligraph
import numpy as np
import pandas as pd
import tensorflow as tf
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.discovery import query_topn, discover_facts, find_clusters
from ampligraph.latent_features import TransE, ComplEx, HolE, DistMult, ConvE, ConvKB
from ampligraph.utils import save_model, restore_model
from tqdm import tqdm

In [58]:
from ampligraph.discovery import find_clusters
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pandas as pd

In [59]:
def display_aggregate_metrics(ranks):
    print('Mean Rank:', mr_score(ranks)) 
    print('Mean Reciprocal Rank:', mrr_score(ranks)) 
    print('Hits@1:', hits_at_n_score(ranks, 1))
    print('Hits@10:', hits_at_n_score(ranks, 10))
    print('Hits@100:', hits_at_n_score(ranks, 100))

print('Ampligraph version: {}'.format(ampligraph.__version__))

Ampligraph version: 1.4.0


In [60]:
import pandas as pd

dataset_new = pd.read_csv("../data/routes.csv")
dataset_new.columns = ['_start', '_end', '_type','_distance']
dataset_new = dataset_new.reindex(columns=['_start', '_type', '_end', '_distance'])
dataset_new.drop(dataset_new.index[dataset_new['_start'] == '_start'], inplace=True)
dataset_new.drop(columns=['_distance'], inplace=True)
dataset_new.head(10)

,_start,_type,_end
0,8627,HAS_ROUTE,8629
1,8627,HAS_ROUTE,8630
2,8627,HAS_ROUTE,8631
3,8627,HAS_ROUTE,8632
4,8627,HAS_ROUTE,8633
5,8627,HAS_ROUTE,8634
6,8627,HAS_ROUTE,8635
7,8627,HAS_ROUTE,8636
8,8627,HAS_ROUTE,8637
9,8627,HAS_ROUTE,8638


In [61]:
ent_to_id ,rel_to_idx = ampligraph.evaluation.create_mappings(dataset_new.to_numpy())

In [62]:
ampligraph.evaluation.to_idx(dataset_new.to_numpy(), rel_to_idx, ent_to_id)

array([[   0,    0,    2],
       [   0,    0,    3],
       [   0,    0,    4],
       ...,
       [8386,    4, 8623],
       [8387,    4, 8622],
       [8388,    4, 8622]])

In [63]:
from ampligraph.evaluation import train_test_split_no_unseen
def test_train_split(dataset):

    test_train, X_valid = train_test_split_no_unseen(dataset.values, 1000, seed=0)
    
    X_train, X_test = train_test_split_no_unseen(test_train, 2000, seed=0)

    print('Total triples:', dataset_new.shape)
    print('Size of train:', X_train.shape)
    print('Size of valid:', X_valid.shape)
    print('Size of test:', X_test.shape)
    return X_train, X_test, X_valid

In [64]:
X_train, X_test, X_valid = test_train_split(dataset_new)

Total triples: (73954, 3)
Size of train: (70954, 3)
Size of valid: (1000, 3)
Size of test: (2000, 3)


In [80]:
X_test

array([[9436, 'HAS_ROUTE', 9487],
       [9567, 'HAS_ROUTE', 8637],
       [16588, 'ON_CONTINENT', 17250],
       ...,
       [8809, 'HAS_ROUTE', 8640],
       [13276, 'IN_REGION', 16136],
       [8889, 'HAS_ROUTE', 8646]], dtype=object)

In [66]:
test_df = pd.DataFrame(np.concatenate((X_test,X_valid)), columns = ['subject','predicate','object'])
test_df.drop(columns=test_df.columns[0], axis=1,  inplace=True)
test_df.to_csv("../data/testdata.csv")

In [67]:
def get_data_for_predicate(predicate, data_location, ret):
    # get dataset
    dataset = fetch_data(data_location)
    
    
    data_with_predicate = dataset.loc[dataset['_type'] == predicate]
    data_with_predicate.reset_index(inplace=True)
    unique_objects = data_with_predicate.object.unique()
    unique_subjects = data_with_predicate.subject.unique()
    if (ret == "sub"):
        return data_with_predicate, unique_subjects
    if (ret == "obj"):
        return data_with_predicate, unique_objects

def find_top_score_item_from_pred(subject, predicate, data_location, model_location):
    dataset = fetch_data(data_location)
    model = restore_model(model_location)
    
    data_for_predicate = dataset.loc[dataset['_type'] == predicate]
    unique_object = data_for_predicate.object.unique()
    
    hypothesis = np.column_stack([[subject] * len(unique_object), 
                              [predicate] * len(unique_object),
                              unique_object,
                             ])
    triple_scores = model.predict(hypothesis);
    scored_hypothesis = np.column_stack([hypothesis, triple_scores]);
    scored_hypothesis = scored_hypothesis[np.argsort(scored_hypothesis[:, 3])];
    max_score_obj = unique_object[np.where(triple_scores == max(triple_scores))[0]];
    return scored_hypothesis[-1]

def predict_obj(model ,pred, sub, top):
    triples, scores = query_topn(model, top_n=top, 
                             head=sub, 
                             relation=pred, 
                             tail=None, 
                             ents_to_consider=None, 
                             rels_to_consider=None,
                            );
    return triples, scores

def predict_sub(model ,pred, obj, top):
    triples, scores = query_topn(model, top_n=top, 
                             head=None, 
                             relation=pred, 
                             tail=obj, 
                             ents_to_consider=None, 
                             rels_to_consider=None,
                            );
    return triples, scores

def triple_in_og_data(data_location, triple):
    dataset = fetch_data(data_location)
    if (len(dataset.loc[(dataset['_start'] == triple[0]) & (dataset['_type'] == triple[1]) & (dataset['_end'] == triple[2])]) > 0 ):
        return True
    else:
        return False
def fetch_data(data_location):
    dataset_new = pd.read_csv("../data/routes.csv")
    dataset_new.columns = ['_start', '_end', '_type','_distance']
    dataset_new = dataset_new.reindex(columns=['_start', '_type', '_end', '_distance'])
    dataset_new.drop(dataset_new.index[dataset_new['_start'] == '_start'], inplace=True)
    dataset_new.drop(columns=['_distance'], inplace=True)
    return dataset_new
def test_train_split(dataset):
    test_train, X_valid = train_test_split_no_unseen(dataset_new.values, 1000, seed=0)

    X_train, X_test = train_test_split_no_unseen(test_train, 2000, seed=0)


    print('Total triples:', dataset.shape)
    print('Size of train:', X_train.shape)
    print('Size of valid:', X_valid.shape)
    print('Size of test:', X_test.shape)
    return X_train, X_test, X_valid

In [103]:
def find_correlation_test_data(model_location):
    model = restore_model(model_location)

    # dataset = fetch_data(data_location)

    X_train, X_test, X_valid = test_train_split(dataset_new)
    
    data_to_test = pd.DataFrame(X_test, columns = ['_start', '_type', '_end'])
    print(data_to_test)

    # print(data_to_test)
    uniuqe_predicate = data_to_test._type.unique()
    
    print(uniuqe_predicate)
    for predicate in uniuqe_predicate:
        correct_triplet = 0
        data_for_predicate = data_to_test.loc[data_to_test['_type'] == predicate]
        data_for_predicate.reset_index(inplace=True)

        print()
        print("\n For Predicate: " + predicate)
        print(data_for_predicate)
        r = 0
        
        for i in range(len(data_for_predicate)):
            print(data_for_predicate.loc[i,"_start"])
            triples, scores = query_topn(model, top_n=1, 
                                         head=9436, 
                                         relation="HAS_ROUTE", 
                                         tail=None, 
                                         ents_to_consider=None, 
                                         rels_to_consider=None
                                         )
            # if (len(data_to_test.loc[(data_to_test['_start'] == triples[0]) & (data_to_test['_type'] == triples[1]) & (data_to_test['_end'] == triples[2])]) > 0 ):
            #     correct_triplet += 1
            break
        # print("Correct Instances Found: " + str(correct_triplet), end =", ")
        # print("Total Instance: " + str(len(data_for_predicate)), end = ", ")
        # print("Accuracy %: " + str(correct_triplet / len(data_for_predicate) * 100))
                  

In [101]:
find_correlation_test_data("Models_new/TransE.pkl")

Total triples: (73954, 3)
Size of train: (70954, 3)
Size of valid: (1000, 3)
Size of test: (2000, 3)
     _start         _type   _end
0      9436     HAS_ROUTE   9487
1      9567     HAS_ROUTE   8637
2     16588  ON_CONTINENT  17250
3      9214     HAS_ROUTE   8642
4      8748     HAS_ROUTE   8694
...     ...           ...    ...
1995  10764     HAS_ROUTE   9145
1996   8820     HAS_ROUTE   8664
1997   8809     HAS_ROUTE   8640
1998  13276     IN_REGION  16136
1999   8889     HAS_ROUTE   8646

[2000 rows x 3 columns]
['HAS_ROUTE' 'ON_CONTINENT' 'IN_COUNTRY' 'IN_REGION' 'IN_CITY']


 For Predicate: HAS_ROUTE
      index _start      _type  _end
0         0   9436  HAS_ROUTE  9487
1         1   9567  HAS_ROUTE  8637
2         3   9214  HAS_ROUTE  8642
3         4   8748  HAS_ROUTE  8694
4         5   8853  HAS_ROUTE  8640
...     ...    ...        ...   ...
1275   1994   9858  HAS_ROUTE  9093
1276   1995  10764  HAS_ROUTE  9145
1277   1996   8820  HAS_ROUTE  8664
1278   1997   8809  HAS_RO

ValueError: Input triples include one or more entities not present in the training set. Please filter all concepts in X that do not occur in the training test (set filter_unseen=True in evaluate_performance) or retrain the model on a training set that includes all the desired concept types.

In [91]:
data_to_train = pd.DataFrame(X_train, columns = ['_start', '_type', '_end'])
data_to_test = pd.DataFrame(X_test, columns = ['_start', '_type', '_end'])
data_to_test.loc[data_to_test['_start'] == 9436]

,_start,_type,_end
0,9436,HAS_ROUTE,9487
606,9436,HAS_ROUTE,8690
1068,9436,HAS_ROUTE,11341
1420,9436,IN_CITY,12895
1964,9436,HAS_ROUTE,8911


In [79]:
model = TransE(k=200,                                                             # embedding size
               epochs=300,                                                        # Num of epochs
               batches_count= 10,                                                 # Number of batches 
               eta=2,                                                             # number of corruptions to generate during training
               loss='pairwise', loss_params={'margin': 1},                        # loss type and it's hyperparameters         
               initializer='xavier', initializer_params={'uniform': False},       # initializer type and it's hyperparameters
               regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},   # regularizer along with its hyperparameters
               optimizer= 'adam', optimizer_params= {'lr': 0.001},                # optimizer to use along with its                
               seed= 0, verbose=True)

model.fit(X_train)

X_filter = np.concatenate([X_train, X_valid, X_test], 0)

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter, filter_unseen=True)

save_model(model, 'Models_new/TransE.pkl')
display_aggregate_metrics(ranks)






100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:24<00:00, 81.04it/s]

Mean Rank: 163.081
Mean Reciprocal Rank: 0.18552176422303676
Hits@1: 0.0005
Hits@10: 0.542
Hits@100: 0.8315


In [31]:
model = ComplEx(k=200, epochs=300, eta=1, loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.0001, 'p': 3},

                
                
                
                seed= 0, batches_count= 1, verbose=True)
model.fit(X_train)

X_filter = np.concatenate([X_train, X_valid, X_test], 0)

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter)

save_model(model, 'Models_new/ComplEx.pkl')
display_aggregate_metrics(ranks)


WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


100%|█████████████████████████████████████████████████████████████████████| 2000/2000 [01:56<00:00, 17.20it/s]

Mean Rank: 213.251
Mean Reciprocal Rank: 0.2387062421129227
Hits@1: 0.1735
Hits@10: 0.3635
Hits@100: 0.67525


In [32]:
model = DistMult(k=200, epochs=300, eta=1, loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.0001, 'p': 3},

                
                
                
                seed= 0, batches_count= 1, verbose=True)
model.fit(X_train)

X_filter = np.concatenate([X_train, X_valid, X_test], 0)

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter)

save_model(model, 'Models_new/DistMult.pkl')
display_aggregate_metrics(ranks)


WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


100%|█████████████████████████████████████████████████████████████████████| 2000/2000 [00:22<00:00, 87.16it/s]

Mean Rank: 230.1245
Mean Reciprocal Rank: 0.2541328064527293
Hits@1: 0.191
Hits@10: 0.3725
Hits@100: 0.65575


In [34]:
model = ConvKB(k=200, epochs=100, eta=1, loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.0001, 'p': 3},
                optimizer= 'adam', optimizer_params= {'lr': 0.001}, 
                seed= 0, 
                batches_count= 5, # Goes OOM (ResourceExhaustedError) if batch count is 1
                verbose=True)
model.fit(X_train)

X_filter = np.concatenate([X_train, X_valid, X_test], 0)

ranks = evaluate_performance(X_test, 
                             model=model, 
                             filter_triples=X_filter)

save_model(model, 'Models_new/ConvKB.pkl')
display_aggregate_metrics(ranks)

100%|█████████████████████████████████████████████████████████████████████| 2000/2000 [25:24<00:00,  1.31it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'Models_old/ConvKB.pkl'

In [35]:
save_model(model, 'Models_new/ConvKB.pkl')
display_aggregate_metrics(ranks)

Mean Rank: 236.736
Mean Reciprocal Rank: 0.22788252786336363
Hits@1: 0.13525
Hits@10: 0.405
Hits@100: 0.74075


In [80]:
find_correlation_test_data("../data/routes.csv", "Models_new/TransE.pkl")

Total triples: (73954, 3)
Size of train: (70954, 3)
Size of valid: (1000, 3)
Size of test: (2000, 3)
     _start         _type   _end
0      9436     HAS_ROUTE   9487
1      9567     HAS_ROUTE   8637
2     16588  ON_CONTINENT  17250
3      9214     HAS_ROUTE   8642
4      8748     HAS_ROUTE   8694
...     ...           ...    ...
1995  10764     HAS_ROUTE   9145
1996   8820     HAS_ROUTE   8664
1997   8809     HAS_ROUTE   8640
1998  13276     IN_REGION  16136
1999   8889     HAS_ROUTE   8646

[2000 rows x 3 columns]
['HAS_ROUTE' 'ON_CONTINENT' 'IN_COUNTRY' 'IN_REGION' 'IN_CITY']
     _start      _type  _end
0      9436  HAS_ROUTE  9487
1      9567  HAS_ROUTE  8637
3      9214  HAS_ROUTE  8642
4      8748  HAS_ROUTE  8694
5      8853  HAS_ROUTE  8640
...     ...        ...   ...
1994   9858  HAS_ROUTE  9093
1995  10764  HAS_ROUTE  9145
1996   8820  HAS_ROUTE  8664
1997   8809  HAS_ROUTE  8640
1999   8889  HAS_ROUTE  8646

[1280 rows x 3 columns]


 For Predicate: HAS_ROUTE
     _start  

ValueError: Input triples include one or more entities not present in the training set. Please filter all concepts in X that do not occur in the training test (set filter_unseen=True in evaluate_performance) or retrain the model on a training set that includes all the desired concept types.